# **PSO ENFOCADO EN AJUSTE DE PARAMETROS**
- 20230033 MOISES RAMIREZ MARTINEZ

In [1]:
# Importaciones
import numpy as np
import pandas as pd
import os
import xlsxwriter  # Para guardar archivos Excel
import time
from openpyxl.styles import PatternFill 

# Importar problemas CEC desde el repositorio
from problemascec2010 import (
    ProblemaC01, ProblemaC02, ProblemaC03, ProblemaC04, ProblemaC05,
    ProblemaC06, ProblemaC07, ProblemaC08, ProblemaC09, ProblemaC10,
    ProblemaC11, ProblemaC12, ProblemaC13, ProblemaC14, ProblemaC15,
    ProblemaC16, ProblemaC17, ProblemaC18
)


# **Manejo de Limites: Reflection**

In [2]:
def manejo_limite_reflection(individuo, lower_bounds, upper_bounds):
    individuo = np.array(individuo)
    lower_bounds = np.array(lower_bounds)
    upper_bounds = np.array(upper_bounds)

    r = upper_bounds - lower_bounds
    valorNorm = (individuo - lower_bounds)

    s = np.floor(valorNorm / r)
    posicion_en_segmento = valorNorm - s * r

    individuo_reflejado = np.zeros_like(individuo)

    cond_par = (s % 2 == 0)

    individuo_reflejado[cond_par] = lower_bounds[cond_par] + posicion_en_segmento[cond_par]

    individuo_reflejado[~cond_par] = upper_bounds[~cond_par] - posicion_en_segmento[~cond_par]

    return individuo_reflejado

# Función que verifica si un individuo está dentro del espacio de búsqueda
def es_valido(individuo, lower_bounds, upper_bounds):
    return np.all((individuo >= lower_bounds) & (individuo <= upper_bounds))

def corregir_individuo(individuo, velocidad, lower_bounds, upper_bounds):
    if not es_valido(individuo, lower_bounds, upper_bounds):
        return manejo_limite_reflection(individuo, lower_bounds, upper_bounds), velocidad
    return individuo, velocidad


# **Manejo de Ajuste de Parametros**

In [3]:
# METODO PROPUESTO
def constriccion_progresiva(gen, max_gen):
    progreso = gen / max_gen
    c1 = 2.5 - progreso * (2.5 - 1.0)
    c2 = 0.5 + progreso * (2.0 - 0.5)
    phi_inicial = 4.1
    phi_incremento = 0.5
    phi = phi_inicial + progreso * phi_incremento
    raiz = np.sqrt(phi ** 2 - 4 * phi)
    w = 2 / abs(2 - phi - raiz)
    return w, c1, c2
    
# 2. Clerc
def clerc(gen=None, max_gen=None):
    c1 = c2 = 2.05
    sumAceleracion = c1 + c2
    x = 2 / abs(2 - sumAceleracion - np.sqrt(sumAceleracion**2 - 4 * sumAceleracion))
    w = x
    return w, c1, c2

# 3. Sigmoid Decreasing Inertia Weight
def sigmoide_decreciente(gen, max_gen):
    w_start = 0.9
    w_end = 0.4

    # Cálculo de u conforme a la fórmula de la imagen
    u = 10 ** (np.log10(max_gen) - 2)

    # k = gen, n = 1 -> fórmula: (k - n * max_gen) = (gen - max_gen)
    exponent = -u * (gen - max_gen)
    w = (w_start - w_end) / (1 + np.exp(exponent)) + w_end

    # Coeficientes constantes
    c1 = 2.0
    c2 = 2.0
    return w, c1, c2

# 4. Linear Decreasing Inertia Weight
def lineal_decreciente(gen, max_gen):
    w_max = 0.9
    w_min = 0.4
    w = w_max - ((w_max - w_min) / max_gen) * gen
    c1 = 2.0
    c2 = 2.0
    return w, c1, c2

# **Funciones de PSO**

In [4]:
def generar_individuo_normal(D, lower, upper):
    media = (lower + upper) / 2
    sigma = (upper - lower) / 6
    return np.random.normal(loc=media, scale=sigma, size=D)

def generar_poblacion(tamano, D, lower_val, upper_val):
    poblacion = np.array([generar_individuo_normal(D, lower_val, upper_val) for _ in range(tamano)])
    return poblacion

def generar_velocidades(tamano, D, lower_val, upper_val):
    velocidades = np.array([generar_individuo_normal(D, lower_val, upper_val) for _ in range(tamano)])
    return velocidades

def actualizar_velocidad_individual(velocidad, posicion, pbest_pos, gbest_pos, w_inercia, c1_cognitivo, c2_social):
    num_dims = len(posicion)
    r1 = np.random.rand(num_dims)
    r2 = np.random.rand(num_dims)
    nueva_velocidad = (
        w_inercia * velocidad +
        c1_cognitivo * r1 * (pbest_pos - posicion) +
        c2_social * r2 * (gbest_pos - posicion)
    )
    return nueva_velocidad

def evaluar_poblacion(poblacion, evaluate_fn):
    resultados = np.array([evaluate_fn(ind) for ind in poblacion])
    fitness_array = resultados[:, 0]
    violaciones_array = resultados[:, 1]
    return fitness_array, violaciones_array

def actualizar_todas_las_velocidades(posiciones, velocidades, pbest_pos, gbest_pos, w, c1, c2):
    nuevas_velocidades = []
    for vel, pos, pbest in zip(velocidades, posiciones, pbest_pos):
        nueva_vel = actualizar_velocidad_individual(vel, pos, pbest, gbest_pos, w, c1, c2)
        nuevas_velocidades.append(nueva_vel)
    return np.array(nuevas_velocidades)

def actualizar_todas_las_posiciones(posiciones, velocidades, lower_bounds, upper_bounds):
    nuevas_posiciones = []
    nuevas_velocidades = []
    for pos, vel in zip(posiciones, velocidades):
        nueva_pos = pos + vel
        nueva_pos, nueva_vel = corregir_individuo(nueva_pos, vel, lower_bounds, upper_bounds)
        nuevas_posiciones.append(nueva_pos)
        nuevas_velocidades.append(nueva_vel)
    return np.array(nuevas_posiciones), np.array(nuevas_velocidades)

def actualizar_todos_los_pbest(posiciones, fitness_array, violaciones_array,
                                pbest_pos, pbest_fit, pbest_viol):
    for i in range(len(posiciones)):
        pbest_pos[i], pbest_fit[i], pbest_viol[i] = actualizar_pbest_individual(
            posiciones[i], fitness_array[i], violaciones_array[i],
            pbest_pos[i], pbest_fit[i], pbest_viol[i]
        )
    return pbest_pos, pbest_fit, pbest_viol

def asignar_gbest_deb(poblacion, fitness_array, viol_array):
    ind_best = poblacion[0]
    f_best   = fitness_array[0]
    v_best   = viol_array[0]

    for ind, f, v in zip(poblacion[1:], fitness_array[1:], viol_array[1:]):
        ind_best, f_best, v_best = deb_selection(ind_best, f_best, v_best, ind, f, v)

    return ind_best, f_best, v_best

def actualizar_pbest_individual(posicion, fitness_valor, violation_valor, pbest_pos, pbest_fitness_valor, pbest_violation_valor):
    _, selected_fitness, selected_violation = deb_selection(
        posicion, fitness_valor, violation_valor, pbest_pos, pbest_fitness_valor, pbest_violation_valor
    )
    if selected_fitness == fitness_valor and selected_violation == violation_valor:
        return posicion.copy(), fitness_valor, violation_valor
    else:
        return pbest_pos, pbest_fitness_valor, pbest_violation_valor


# **Reglas de Deb**

In [5]:
def deb_selection(ind1, f1, v1, ind2, f2, v2):
    feas1 = (v1 <= 0)
    feas2 = (v2 <= 0)
    if feas1 and feas2:
        if f1 < f2:
            return ind1, f1, v1
        else:
            return ind2, f2, v2
    elif feas1:
        return ind1, f1, v1
    elif feas2:
        return ind2, f2, v2
    elif v1 <= v2:
        return ind1, f1, v1
    else:
        return ind2, f2, v2

# **Problemas CEC2010**

In [6]:
problems = {
    'C01': ProblemaC01,
    'C02': ProblemaC02,
    'C03': ProblemaC03,
    'C04': ProblemaC04,
    'C05': ProblemaC05,
    'C06': ProblemaC06,
    'C07': ProblemaC07,
    'C08': ProblemaC08,
    'C09': ProblemaC09,
    'C10': ProblemaC10,
    'C11': ProblemaC11,
    'C12': ProblemaC12,
    'C13': ProblemaC13,
    'C14': ProblemaC14,
    'C15': ProblemaC15,
    'C16': ProblemaC16,
    'C17': ProblemaC17,
    'C18': ProblemaC18
}

# **PARAMETROS**
- TAMAÑO DE LA POBLACION
- NUMERO DE GENERACIONES
- NUMERO DE CORRIDAS
- NUMERO DE DIMENSIONES(OFFSET) 

In [7]:
pso_params = {
    'tamano_poblacion': 100,
    'num_generaciones': 2000,
    'num_corridas': 25,
    # 10 Dimensiones o 30 Dimensiones en este caso
    'offset':  np.array([
        1.91, 4.86, 3.74, 3.06, 0.79, 0.79, 0.29, 4.43, 3.07, 3.62
        #,0.58, 4.51, 2.13, 1.45, 3.23, 2.37, 0.69, 4.92, 1.98, 1.56,
        #0.37, 4.77, 3.95, 1.11, 0.24, 2.78, 4.44, 2.65, 0.08, 5.10
    ],dtype=float)
}

In [8]:
ajuste_parametros = {
    'Constriccion Progresiva': constriccion_progresiva,
    'Clerc': clerc,
    'Lineal Decreciente': lineal_decreciente,
    'Sigmoide Decreciente':sigmoide_decreciente
}

# **EJECUCION DE PSO**

In [9]:
def ejecutar_pso(tamano_poblacion, lower_bounds, upper_bounds, evaluate_fn, generaciones, funcion_limite, ajuste_parametros_func):
    D = len(lower_bounds)

    # Inicialización
    posiciones = generar_poblacion(tamano_poblacion, D, lower_bounds, upper_bounds)
    velocidades = generar_velocidades(tamano_poblacion, D, lower_bounds, upper_bounds)

    fitness, violaciones = evaluar_poblacion(posiciones, evaluate_fn)

    pbest_pos = posiciones.copy()
    pbest_fit = fitness.copy()
    pbest_viol = violaciones.copy()

    gbest_pos, gbest_fit, gbest_viol = asignar_gbest_deb(pbest_pos, pbest_fit, pbest_viol)

    mejores_por_generacion = []

    for gen in range(1, generaciones + 1):
        w, c1, c2 = ajuste_parametros_func(gen-1, generaciones)

        # a) Actualizar velocidades
        velocidades = actualizar_todas_las_velocidades(posiciones, velocidades, pbest_pos, gbest_pos, w, c1, c2)

        # b) Actualizar posiciones
        posiciones, velocidades = actualizar_todas_las_posiciones(posiciones, velocidades, lower_bounds, upper_bounds)

        # c) Evaluar
        fitness, violaciones = evaluar_poblacion(posiciones, evaluate_fn)

        # d) Actualizar pbest
        pbest_pos, pbest_fit, pbest_viol = actualizar_todos_los_pbest(posiciones, fitness, violaciones, pbest_pos, pbest_fit, pbest_viol)

        # e) Actualizar gbest
        gbest_pos, gbest_fit, gbest_viol = asignar_gbest_deb(pbest_pos, pbest_fit, pbest_viol)

        # f) Guardar mejor de la generación
        mejores_por_generacion.append({
            'individuo': gbest_pos.copy(),
            'fitness': gbest_fit,
            'violation': gbest_viol,
            'factible': gbest_viol <= 0
        })

    return mejores_por_generacion


In [10]:
def ejecutar_combinaciones_PSO(
    clases_problema,
    nombres_problemas,
    metodos_ajuste,
    nombres_ajuste,
    num_corridas,
    tamano_poblacion,
    offset,
    funcion_limite,
    generaciones
):
    for cls_prob, nombre_prob in zip(clases_problema, nombres_problemas):
        problema = cls_prob(offset)
        lower_bounds, upper_bounds = problema.get_limites()
        evaluate_fn = problema.evaluate

        archivo = f"PSO_{nombre_prob}.xlsx"
        with pd.ExcelWriter(archivo, engine="openpyxl") as writer:
            for metodo_ajuste, nombre_ajuste in zip(metodos_ajuste, nombres_ajuste):
                print(f"\n🔁 Problema = {nombre_prob} | Ajuste = {nombre_ajuste}")

                matriz = np.zeros((generaciones, num_corridas))
                tiempos = []
                factibles_final = []
                violaciones_final = []

                for j in range(num_corridas):
                    inicio = time.time()
                    resultados = ejecutar_pso(
                        tamano_poblacion,
                        lower_bounds,
                        upper_bounds,
                        evaluate_fn,
                        generaciones,
                        funcion_limite,
                        metodo_ajuste
                    )
                    tiempos.append(round(time.time() - inicio, 2))
                    matriz[:, j] = [r['fitness'] for r in resultados]
                    factibles_final.append(resultados[-1]['factible'])
                    violaciones_final.append(resultados[-1]['violation'])

                # DataFrame
                cols = [f"Corrida {i+1}" for i in range(num_corridas)]
                df = pd.DataFrame(matriz, columns=cols)
                df.insert(0, "Generación", np.arange(1, generaciones + 1))
                df["Promedio"] = df[cols].astype(float).mean(axis=1)

                # Agregar filas extra
                df.loc[len(df)] = ["Violaciones"] + violaciones_final + [""]
                df.loc[len(df)] = ["Tiempo (s)"] + tiempos + [""]
                df.loc[len(df)] = ["Factibles"] + [int(f) for f in factibles_final] + [""]

                sheet_name = nombre_ajuste[:31]
                df.to_excel(writer, index=False, sheet_name=sheet_name)

                # Colorear última gen factible
                fila_ultima_gen = generaciones + 1
                amarillo = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
                for col_idx, fact in enumerate(factibles_final, start=2):
                    if fact:
                        writer.sheets[sheet_name].cell(row=fila_ultima_gen, column=col_idx).fill = amarillo

        print(f"\n✅ Archivo guardado: {archivo}")


In [11]:

metodos_ajuste = list(ajuste_parametros.values())
nombres_ajuste = list(ajuste_parametros.keys())

ejecutar_combinaciones_PSO(
    clases_problema=list(problems.values()),
    nombres_problemas=list(problems.keys()),
    metodos_ajuste=metodos_ajuste,
    nombres_ajuste=nombres_ajuste,
    num_corridas=pso_params['num_corridas'],
    tamano_poblacion=pso_params['tamano_poblacion'],
    offset=pso_params['offset'],
    funcion_limite=manejo_limite_reflection,
    generaciones=pso_params['num_generaciones']
)

print("\n✅ Archivos guardados")



🔁 Problema = C01 | Ajuste = Constriccion Progresiva

🔁 Problema = C01 | Ajuste = Clerc

🔁 Problema = C01 | Ajuste = Lineal Decreciente

🔁 Problema = C01 | Ajuste = Sigmoide Decreciente


C:\Users\moyra\AppData\Local\Temp\ipykernel_30216\1264782672.py:24: RuntimeWarning: overflow encountered in exp
  w = (w_start - w_end) / (1 + np.exp(-u * (gen - max_gen / 2))) + w_end



✅ Archivo guardado: PSO_C01.xlsx

🔁 Problema = C02 | Ajuste = Constriccion Progresiva

🔁 Problema = C02 | Ajuste = Clerc

🔁 Problema = C02 | Ajuste = Lineal Decreciente

🔁 Problema = C02 | Ajuste = Sigmoide Decreciente


C:\Users\moyra\AppData\Local\Temp\ipykernel_30216\1264782672.py:24: RuntimeWarning: overflow encountered in exp
  w = (w_start - w_end) / (1 + np.exp(-u * (gen - max_gen / 2))) + w_end



✅ Archivo guardado: PSO_C02.xlsx

🔁 Problema = C03 | Ajuste = Constriccion Progresiva

🔁 Problema = C03 | Ajuste = Clerc

🔁 Problema = C03 | Ajuste = Lineal Decreciente

🔁 Problema = C03 | Ajuste = Sigmoide Decreciente


...


✅ Archivo guardado: PSO_C16.xlsx

🔁 Problema = C17 | Ajuste = Constriccion Progresiva

🔁 Problema = C17 | Ajuste = Clerc

🔁 Problema = C17 | Ajuste = Lineal Decreciente

🔁 Problema = C17 | Ajuste = Sigmoide Decreciente


C:\Users\moyra\AppData\Local\Temp\ipykernel_30216\1264782672.py:24: RuntimeWarning: overflow encountered in exp
  w = (w_start - w_end) / (1 + np.exp(-u * (gen - max_gen / 2))) + w_end



✅ Archivo guardado: PSO_C17.xlsx

🔁 Problema = C18 | Ajuste = Constriccion Progresiva

🔁 Problema = C18 | Ajuste = Clerc

🔁 Problema = C18 | Ajuste = Lineal Decreciente

🔁 Problema = C18 | Ajuste = Sigmoide Decreciente


C:\Users\moyra\AppData\Local\Temp\ipykernel_30216\1264782672.py:24: RuntimeWarning: overflow encountered in exp
  w = (w_start - w_end) / (1 + np.exp(-u * (gen - max_gen / 2))) + w_end



✅ Archivo guardado: PSO_C18.xlsx

✅ Archivos guardados
